In [ ]:
!pip install requests
!pip install tensorflow_hub

Adapted from [this gist](https://gist.github.com/sameerg07/4e0337ed2f77845edbb319633bd324e5/), this first bit takes classification data trained in [this Dataturks project](https://dataturks.com/projects/fried.chris/Destiny%202%20Screen%20Classification), stored in [this gist](https://gist.github.com/chrisfried/75e257f6595f1449e4477c74c07198df/), and outputs it into a folder structure Tensorflow can monch on.

In [ ]:
import json
import requests
import os
import shutil

def downloader(image_url , i , k):
    file_name = i + k
    full_file_name = str(file_name) + '.jpg'
    r = requests.get(image_url)
    with open('dataset/' + i + '/' + full_file_name, 'wb') as fd:
        for chunk in r.iter_content(chunk_size=128):
            fd.write(chunk)
if os.path.exists('d2screens.json'):
	os.remove('d2screens.json')
    
jr = requests.get('https://gist.githubusercontent.com/chrisfried/75e257f6595f1449e4477c74c07198df/raw/d2screens.json')
with open('d2screens.json', 'wb') as fd:
    for chunk in jr.iter_content(chunk_size=128):
        fd.write(chunk)

with open('./d2screens.json') as file1:
	lis = []	
	for i in file1:
		lis.append(json.loads(i))

folder_names = []
label_to_urls = {}

for i in lis:
	try:
		if i['annotation']['labels'][0] not in folder_names:
			folder_names.append(i['annotation']['labels'][0])
			label_to_urls[i['annotation']['labels'][0]] = [i['content']] 	
		else:
			label_to_urls[i['annotation']['labels'][0]].append(i['content'])
	except:
		print('json parse error')

if os.path.isdir('dataset'):
	shutil.rmtree('dataset')
    
os.mkdir('dataset')

for i in label_to_urls.keys():
# 	if str(i) == 'roster' or str(i) == 'player menu' or str(i) == 'pgcr' or str(i) == 'kill screen' or str(i) == 'character select': 
    try:
        os.mkdir('dataset/' + str(i))
    except:
        print('folder exists')
    k = 0;	
    for j in label_to_urls[i]:
        downloader(j , str(i), str(k))
        k+=1
    if k < 20:
        shutil.rmtree('dataset/' + str(i))
        
if os.path.exists('d2screens.json'):
	os.remove('d2screens.json')

Download and retrain the image model

In [ ]:
import shutil

!curl -LO https://github.com/tensorflow/hub/raw/master/examples/image_retraining/retrain.py
!python retrain.py --image_dir ./dataset
shutil.copy('/tmp/output_graph.pb', 'd2_classifier_graph.pb')
shutil.copy('/tmp/output_labels.txt', 'd2_classifier_labels.txt')